User Input
   │
   ▼
[Step 2] → classify intent + extract entities via LLM
   │
   ▼
[Step 3] → build SQL prompt using schema + examples
   │
   ▼
[Step 4] → generate SQL via LLM
   │
   ▼
[Step 5] → validate SQL
   │
   ▼
[Step 6] → optional agentic reflection + fix SQL via LLM
   │
   ▼
[Step 7] → execute SQL on SQLite
   │
   ▼
[Step 8] → return final answer via LLM


In [ ]:
import os
import json
import httpx  # Needed for custom SSL override if you're using it
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=OPENAI_API_KEY,
    http_client=httpx.Client(verify=os.environ.get("REQUESTS_CA_BUNDLE"))
)


NFL_TEAMS = [
    "ARI","ATL","BAL","BUF","CAR","CHI","CIN","CLE","DAL","DEN",
    "DET","GB","HOU","IND","JAX","KC","LV","LAC","LAR","MIA",
    "MIN","NE","NO","NYG","NYJ","PHI","PIT","SEA","SF","TB",
    "TEN","WAS"
]

def classify_intent_and_extract_entities(question: str) -> dict:
    prompt = f"""
You are a classifier and entity extractor for a sports assistant focused on the Philadelphia Eagles.

Your job is to:
1. Classify the question into one of these intents:
   - "game": about a specific game, week, quarter, half, drive, or play
   - "player": about a specific player's stats or performance
   - "season": about full-season team performance or summaries

2. Extract e`ntities into this format:
{{
  "season": integer or null,
  "week": integer or null,
  "player": string or null,
  "opponent": string (NFL abbreviation) or null
}}

Rules:
- The team is always PHI (Eagles) so do NOT extract that.
- Extract opponent ONLY if another team is mentioned.
- If a field is not mentioned in the question, return null.
- Return a single JSON object with "intent" and "entities" as keys.

User question:
{question}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_completion_tokens=200
    )

    raw = response.choices[0].message.content
    print("RAW MODEL OUTPUT:", raw)

    try:
        result = json.loads(raw)

        # Normalize opponent to valid teams only
        if result.get("entities", {}).get("opponent") not in NFL_TEAMS:
            result["entities"]["opponent"] = None

        # Validate intent
        if result.get("intent") not in {"game", "player", "season"}:
            result["intent"] = "season"  # safe fallback

        return result

    except Exception as e:
        print("JSON PARSE ERROR:", e)
        return {
            "intent": "season",  # default
            "entities": {
                "season": None,
                "week": None,
                "player": None,
                "opponent": None
            }
        }



In [5]:
result = classify_intent_and_extract_entities("How many passing yards did Jalen Hurts have in Week 9 of 2024 against Dallas?")
print(json.dumps(result, indent=2))

RAW MODEL OUTPUT: {
  "intent": "player",
  "entities": {
    "season": 2024,
    "week": 9,
    "player": "Jalen Hurts",
    "opponent": "DAL"
  }
}
JSON PARSE ERROR: name 'NFL_TEAMS' is not defined
{
  "intent": "season",
  "entities": {
    "season": null,
    "week": null,
    "player": null,
    "opponent": null
  }
}
